# Aspect Detection Exploration

Created: 2019.1.10  
Updated: 2019.1.10

### _An unsupervised aspect detection model for sentiment analysis of reviews_

It looks like they start with a "seed set" of aspects (the seed set is found unsupervised??)

It iteratively bootstraps (and clusters?) to find better final aspect list

Use a generalized version of an FLR method to "rank aspects and select important ones"  
("FLR is a word scoring method that uses internal structures and frequencies of candidates")

### _An Unsupervised Neural Attention Model for Aspect Extraction_
(Note that there is code for this paper at https://github.com/ruidan/Unsupervised-Aspect-Extraction)

Interestingly they note that LDA is frequently used for aspect extraction (but go on to state why this generally doesn't work very well)

Does this work handle multi-word aspects?